<a href="https://colab.research.google.com/github/maocafe29/cs_9414/blob/main/ass2_9414.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP9414 Artificial Intelligence - Assignment 2: Reinforcement Learning

## Environment Setup

In [ ]:
# Check if running in Google Colab
import os
import sys

try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except:
    IN_COLAB = False
    print("Running locally")

In [ ]:
# File setup for Colab (skip if running locally)
if IN_COLAB:
    from google.colab import files

    # Create directories
    os.makedirs('images', exist_ok=True)

    # Function to check and upload files
    def setup_files():
        required_files = {
            'env.py': '.',
            'grid_agent.png': 'images',
            'grid_goal_position.png': 'images',
            'grid_obstacle.png': 'images'
        }

        for filename, directory in required_files.items():
            filepath = os.path.join(directory, filename) if directory != '.' else filename
            if not os.path.exists(filepath):
                print(f"Please upload {filename}:")
                uploaded = files.upload()
                if filename in uploaded and directory != '.':
                    os.rename(filename, filepath)

    setup_files()

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm
from env import GridWorldEnv

# Set up matplotlib
%matplotlib inline
plt.style.use('default')

# Set random seed for reproducibility
np.random.seed(42)

print("Environment setup complete!")

## Hyperparameters

All tasks will use the same hyperparameters to ensure fair comparison.

In [ ]:
# Global hyperparameters (same for all tasks)
HYPERPARAMETERS = {
    'learning_rate': 0.3,
    'discount_factor': 0.95,
    'epsilon_start': 1.0,
    'epsilon_end': 0.01,
    'epsilon_decay_episodes': 300,
    'num_episodes': 500,
    'max_steps_per_episode': 100,
    'seed': 42
}

# Teacher parameters (as per assignment requirements)
TEACHER_PARAMS = {
    'availability': [0.1, 0.3, 0.5, 0.7, 1.0],
    'accuracy': [0.1, 0.3, 0.5, 0.7, 1.0]
}

print("Hyperparameters:")
for key, value in HYPERPARAMETERS.items():
    print(f"  {key}: {value}")

## Helper Functions

In [ ]:
def calculate_metrics(episode_rewards, episode_steps, successful_episodes):
    """Calculate performance metrics"""
    success_rate = (sum(successful_episodes) / len(successful_episodes)) * 100
    avg_reward = np.mean(episode_rewards)
    avg_steps = np.mean(episode_steps)
    avg_learning_speed = 1 / avg_steps if avg_steps > 0 else 0

    return {
        'success_rate': success_rate,
        'avg_reward': avg_reward,
        'avg_learning_speed': avg_learning_speed
    }

def plot_learning_curve(rewards, title, window=50):
    """Plot learning curve with moving average"""
    plt.figure(figsize=(10, 6))

    # Plot raw rewards with transparency
    plt.plot(rewards, alpha=0.3, color='blue', label='Episode Rewards')

    # Calculate and plot moving average
    if len(rewards) >= window:
        moving_avg = pd.Series(rewards).rolling(window=window).mean()
        plt.plot(moving_avg, color='darkblue', linewidth=2,
                label=f'{window}-episode MA')

    # Add reference line at y=0
    plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)

    plt.xlabel('Episode')
    plt.ylabel('Reward per Episode')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

def create_heatmap(results_df, algorithm_name):
    """Create heatmap for teacher experiment results"""
    # Pivot data for heatmap
    pivot_table = results_df.pivot(index='accuracy',
                                  columns='availability',
                                  values='avg_reward')

    # Create heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(pivot_table, annot=True, fmt='.1f', cmap='RdYlBu_r',
                cbar_kws={'label': 'Average Reward'})
    plt.title(f'{algorithm_name} with Teacher - Average Reward Heatmap')
    plt.xlabel('Teacher Availability')
    plt.ylabel('Teacher Accuracy')
    plt.show()

# Task 1: Implement Q-learning

In this task, we implement the Q-learning algorithm and train an agent in the grid world environment.

In [ ]:
class QLearningAgent:
    """Q-learning agent implementation"""

    def __init__(self, env, learning_rate=0.3, discount_factor=0.95,
                 epsilon_start=1.0, epsilon_end=0.01, epsilon_decay_episodes=300):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay_episodes = epsilon_decay_episodes

        # Initialize Q-table
        self.q_table = np.zeros((env.grid_size, env.grid_size, env.action_space))

    def get_epsilon(self, episode):
        """Calculate epsilon value for current episode (linear decay)"""
        if episode >= self.epsilon_decay_episodes:
            return self.epsilon_end
        return self.epsilon_start - (self.epsilon_start - self.epsilon_end) * \
               (episode / self.epsilon_decay_episodes)

    def choose_action(self, state, epsilon):
        """Epsilon-greedy action selection"""
        if np.random.uniform(0, 1) < epsilon:
            return np.random.randint(self.env.action_space)
        else:
            return np.argmax(self.q_table[state[0], state[1]])

    def update(self, state, action, reward, next_state, done):
        """Q-learning update rule"""
        current_q = self.q_table[state[0], state[1], action]

        if done:
            target = reward
        else:
            # Q-learning: use maximum Q-value of next state
            max_next_q = np.max(self.q_table[next_state[0], next_state[1]])
            target = reward + self.discount_factor * max_next_q

        # Update Q-value
        self.q_table[state[0], state[1], action] = \
            current_q + self.learning_rate * (target - current_q)

In [ ]:
def train_q_learning(env, agent, num_episodes):
    """Train Q-learning agent"""
    episode_rewards = []
    episode_steps = []
    successful_episodes = []

    for episode in tqdm(range(num_episodes), desc="Training Q-learning"):
        state = env.reset()
        done = False
        total_reward = 0
        steps = 0
        epsilon = agent.get_epsilon(episode)

        while not done and steps < HYPERPARAMETERS['max_steps_per_episode']:
            # Choose action
            action = agent.choose_action(state, epsilon)

            # Take action
            next_state, reward, done, _ = env.step(action)

            # Update Q-value
            agent.update(state, action, reward, next_state, done)

            # Update state and statistics
            state = next_state
            total_reward += reward
            steps += 1

        # Record metrics
        episode_rewards.append(total_reward)
        episode_steps.append(steps)
        successful_episodes.append(1 if done and reward == 25 else 0)

    return episode_rewards, episode_steps, successful_episodes

In [ ]:
# Create environment
env = GridWorldEnv(seed=HYPERPARAMETERS['seed'])

# Create and train Q-learning agent
print("Training Q-learning agent...")
q_agent = QLearningAgent(env, **{k: v for k, v in HYPERPARAMETERS.items()
                                if k not in ['num_episodes', 'max_steps_per_episode', 'seed']})

q_rewards, q_steps, q_success = train_q_learning(env, q_agent, HYPERPARAMETERS['num_episodes'])

# Calculate metrics
q_metrics = calculate_metrics(q_rewards, q_steps, q_success)

print("\nQ-learning Results:")
print(f"Success Rate: {q_metrics['success_rate']:.2f}%")
print(f"Average Reward: {q_metrics['avg_reward']:.2f}")
print(f"Average Learning Speed: {q_metrics['avg_learning_speed']:.4f}")

In [ ]:
# Plot Q-learning performance
plot_learning_curve(q_rewards, "Q-learning Performance")

# Task 2: Implement SARSA

In this task, we implement the SARSA algorithm using the same hyperparameters as Task 1 for fair comparison.

In [ ]:
class SARSAAgent:
    """SARSA agent implementation"""

    def __init__(self, env, learning_rate=0.3, discount_factor=0.95,
                 epsilon_start=1.0, epsilon_end=0.01, epsilon_decay_episodes=300):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay_episodes = epsilon_decay_episodes

        # Initialize Q-table
        self.q_table = np.zeros((env.grid_size, env.grid_size, env.action_space))

    def get_epsilon(self, episode):
        """Calculate epsilon value for current episode (linear decay)"""
        if episode >= self.epsilon_decay_episodes:
            return self.epsilon_end
        return self.epsilon_start - (self.epsilon_start - self.epsilon_end) * \
               (episode / self.epsilon_decay_episodes)

    def choose_action(self, state, epsilon):
        """Epsilon-greedy action selection"""
        if np.random.uniform(0, 1) < epsilon:
            return np.random.randint(self.env.action_space)
        else:
            return np.argmax(self.q_table[state[0], state[1]])

    def update(self, state, action, reward, next_state, next_action, done):
        """SARSA update rule"""
        current_q = self.q_table[state[0], state[1], action]

        if done:
            target = reward
        else:
            # SARSA: use Q-value of next state-action pair
            next_q = self.q_table[next_state[0], next_state[1], next_action]
            target = reward + self.discount_factor * next_q

        # Update Q-value
        self.q_table[state[0], state[1], action] = \
            current_q + self.learning_rate * (target - current_q)

In [ ]:
def train_sarsa(env, agent, num_episodes):
    """Train SARSA agent"""
    episode_rewards = []
    episode_steps = []
    successful_episodes = []

    for episode in tqdm(range(num_episodes), desc="Training SARSA"):
        state = env.reset()
        epsilon = agent.get_epsilon(episode)
        action = agent.choose_action(state, epsilon)
        done = False
        total_reward = 0
        steps = 0

        while not done and steps < HYPERPARAMETERS['max_steps_per_episode']:
            # Take action
            next_state, reward, done, _ = env.step(action)

            # Choose next action
            next_action = agent.choose_action(next_state, epsilon)

            # Update Q-value
            agent.update(state, action, reward, next_state, next_action, done)

            # Update state and action
            state = next_state
            action = next_action
            total_reward += reward
            steps += 1

        # Record metrics
        episode_rewards.append(total_reward)
        episode_steps.append(steps)
        successful_episodes.append(1 if done and reward == 25 else 0)

    return episode_rewards, episode_steps, successful_episodes

In [ ]:
# Create and train SARSA agent
print("Training SARSA agent...")
sarsa_agent = SARSAAgent(env, **{k: v for k, v in HYPERPARAMETERS.items()
                                if k not in ['num_episodes', 'max_steps_per_episode', 'seed']})

sarsa_rewards, sarsa_steps, sarsa_success = train_sarsa(env, sarsa_agent, HYPERPARAMETERS['num_episodes'])

# Calculate metrics
sarsa_metrics = calculate_metrics(sarsa_rewards, sarsa_steps, sarsa_success)

print("\nSARSA Results:")
print(f"Success Rate: {sarsa_metrics['success_rate']:.2f}%")
print(f"Average Reward: {sarsa_metrics['avg_reward']:.2f}")
print(f"Average Learning Speed: {sarsa_metrics['avg_learning_speed']:.4f}")

In [ ]:
# Plot SARSA performance
plot_learning_curve(sarsa_rewards, "SARSA Performance")

## Baseline Comparison

Compare the performance of Q-learning and SARSA without teacher guidance.

In [ ]:
# Create comparison plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Learning progress comparison
ax1.plot(q_rewards, alpha=0.3, color='blue', label='Q-learning')
ax1.plot(sarsa_rewards, alpha=0.3, color='green', label='SARSA')

# Add moving averages
window = 50
if len(q_rewards) >= window:
    q_ma = pd.Series(q_rewards).rolling(window=window).mean()
    sarsa_ma = pd.Series(sarsa_rewards).rolling(window=window).mean()
    ax1.plot(q_ma, color='darkblue', linewidth=2, label=f'Q-learning MA({window})')
    ax1.plot(sarsa_ma, color='darkgreen', linewidth=2, label=f'SARSA MA({window})')

ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax1.set_xlabel('Episode')
ax1.set_ylabel('Reward per Episode')
ax1.set_title('Learning Progress Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Success rate comparison
window = 50
q_success_rate = pd.Series(q_success).rolling(window=window).mean() * 100
sarsa_success_rate = pd.Series(sarsa_success).rolling(window=window).mean() * 100

ax2.plot(q_success_rate, color='blue', label=f'Q-learning ({q_metrics["success_rate"]:.1f}% overall)')
ax2.plot(sarsa_success_rate, color='green', label=f'SARSA ({sarsa_metrics["success_rate"]:.1f}% overall)')
ax2.set_xlabel('Episode')
ax2.set_ylabel('Success Rate (%)')
ax2.set_title('Success Rate Comparison')
ax2.set_ylim(0, 105)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Task 3: Q-learning with Teacher

Implement teacher-student framework where a pre-trained Q-learning agent guides a new Q-learning student.

In [ ]:
def teacher_guided_action(state, student_q_table, teacher_q_table, epsilon,
                         availability, accuracy, env):
    """Select action with teacher guidance"""
    # Check if teacher provides advice
    if np.random.uniform(0, 1) < availability:
        # Teacher provides advice
        teacher_best_action = np.argmax(teacher_q_table[state[0], state[1]])

        # Check if advice is correct
        if np.random.uniform(0, 1) < accuracy:
            # Correct advice: use teacher's best action
            return teacher_best_action
        else:
            # Incorrect advice: random action excluding teacher's best
            possible_actions = list(range(env.action_space))
            possible_actions.remove(teacher_best_action)
            return np.random.choice(possible_actions)
    else:
        # No teacher advice: use student's epsilon-greedy policy
        if np.random.uniform(0, 1) < epsilon:
            return np.random.randint(env.action_space)
        else:
            return np.argmax(student_q_table[state[0], state[1]])

In [ ]:
def train_q_learning_with_teacher(env, teacher_q_table, num_episodes,
                                 availability, accuracy, hyperparams):
    """Train Q-learning with teacher guidance"""
    # Create new student agent
    student = QLearningAgent(env, **{k: v for k, v in hyperparams.items()
                                    if k not in ['num_episodes', 'max_steps_per_episode', 'seed']})

    episode_rewards = []
    episode_steps = []
    successful_episodes = []

    for episode in range(num_episodes):
        state = env.reset()
        done = False
        total_reward = 0
        steps = 0
        epsilon = student.get_epsilon(episode)

        while not done and steps < hyperparams['max_steps_per_episode']:
            # Choose action with teacher guidance
            action = teacher_guided_action(state, student.q_table, teacher_q_table,
                                         epsilon, availability, accuracy, env)

            # Take action
            next_state, reward, done, _ = env.step(action)

            # Update Q-value
            student.update(state, action, reward, next_state, done)

            # Update state and statistics
            state = next_state
            total_reward += reward
            steps += 1

        # Record metrics
        episode_rewards.append(total_reward)
        episode_steps.append(steps)
        successful_episodes.append(1 if done and reward == 25 else 0)

    return episode_rewards, episode_steps, successful_episodes

In [ ]:
# Run Q-learning with teacher experiments
print("Running Q-learning with teacher experiments...")
print("This will test 25 different teacher configurations.\n")

q_teacher_results = []

for avail in TEACHER_PARAMS['availability']:
    for acc in TEACHER_PARAMS['accuracy']:
        print(f"Testing availability={avail}, accuracy={acc}")

        # Train with teacher
        rewards, steps, success = train_q_learning_with_teacher(
            env, q_agent.q_table,
            num_episodes=300,  # Can use fewer episodes for teacher experiments
            availability=avail,
            accuracy=acc,
            hyperparams=HYPERPARAMETERS
        )

        # Calculate metrics
        metrics = calculate_metrics(rewards, steps, success)

        # Store results
        q_teacher_results.append({
            'availability': avail,
            'accuracy': acc,
            'avg_reward': metrics['avg_reward'],
            'success_rate': metrics['success_rate'],
            'avg_learning_speed': metrics['avg_learning_speed']
        })

        print(f"  Average Reward: {metrics['avg_reward']:.2f}")
        print(f"  Success Rate: {metrics['success_rate']:.1f}%\n")

In [ ]:
# Create heatmap for Q-learning with teacher
q_teacher_df = pd.DataFrame(q_teacher_results)
create_heatmap(q_teacher_df, "Q-learning")

# Task 4: SARSA with Teacher

Implement teacher-student framework where a pre-trained SARSA agent guides a new SARSA student.

In [ ]:
def train_sarsa_with_teacher(env, teacher_q_table, num_episodes,
                            availability, accuracy, hyperparams):
    """Train SARSA with teacher guidance"""
    # Create new student agent
    student = SARSAAgent(env, **{k: v for k, v in hyperparams.items()
                                if k not in ['num_episodes', 'max_steps_per_episode', 'seed']})

    episode_rewards = []
    episode_steps = []
    successful_episodes = []

    for episode in range(num_episodes):
        state = env.reset()
        epsilon = student.get_epsilon(episode)

        # Choose initial action with teacher guidance
        action = teacher_guided_action(state, student.q_table, teacher_q_table,
                                     epsilon, availability, accuracy, env)

        done = False
        total_reward = 0
        steps = 0

        while not done and steps < hyperparams['max_steps_per_episode']:
            # Take action
            next_state, reward, done, _ = env.step(action)

            # Choose next action with teacher guidance
            next_action = teacher_guided_action(next_state, student.q_table, teacher_q_table,
                                              epsilon, availability, accuracy, env)

            # Update Q-value
            student.update(state, action, reward, next_state, next_action, done)

            # Update state and action
            state = next_state
            action = next_action
            total_reward += reward
            steps += 1

        # Record metrics
        episode_rewards.append(total_reward)
        episode_steps.append(steps)
        successful_episodes.append(1 if done and reward == 25 else 0)

    return episode_rewards, episode_steps, successful_episodes

In [ ]:
# Run SARSA with teacher experiments
print("Running SARSA with teacher experiments...")
print("This will test 25 different teacher configurations.\n")

sarsa_teacher_results = []

for avail in TEACHER_PARAMS['availability']:
    for acc in TEACHER_PARAMS['accuracy']:
        print(f"Testing availability={avail}, accuracy={acc}")

        # Train with teacher
        rewards, steps, success = train_sarsa_with_teacher(
            env, sarsa_agent.q_table,
            num_episodes=300,  # Can use fewer episodes for teacher experiments
            availability=avail,
            accuracy=acc,
            hyperparams=HYPERPARAMETERS
        )

        # Calculate metrics
        metrics = calculate_metrics(rewards, steps, success)

        # Store results
        sarsa_teacher_results.append({
            'availability': avail,
            'accuracy': acc,
            'avg_reward': metrics['avg_reward'],
            'success_rate': metrics['success_rate'],
            'avg_learning_speed': metrics['avg_learning_speed']
        })

        print(f"  Average Reward: {metrics['avg_reward']:.2f}")
        print(f"  Success Rate: {metrics['success_rate']:.1f}%\n")

In [ ]:
# Create heatmap for SARSA with teacher
sarsa_teacher_df = pd.DataFrame(sarsa_teacher_results)
create_heatmap(sarsa_teacher_df, "SARSA")

# Analysis and Discussion

## Teacher Impact Analysis

In [ ]:
# Analyze teacher impact for specific availability levels
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Q-learning analysis
for avail in [0.1, 0.5, 1.0]:
    df_subset = q_teacher_df[q_teacher_df['availability'] == avail]
    ax1.plot(df_subset['accuracy'], df_subset['avg_reward'],
             marker='o', label=f'Availability={avail}')

ax1.axhline(y=q_metrics['avg_reward'], color='red', linestyle='--',
           label='Baseline (no teacher)')
ax1.set_xlabel('Teacher Accuracy')
ax1.set_ylabel('Average Reward')
ax1.set_title('Q-learning: Teacher Impact Analysis')
ax1.legend()
ax1.grid(True, alpha=0.3)

# SARSA analysis
for avail in [0.1, 0.5, 1.0]:
    df_subset = sarsa_teacher_df[sarsa_teacher_df['availability'] == avail]
    ax2.plot(df_subset['accuracy'], df_subset['avg_reward'],
             marker='o', label=f'Availability={avail}')

ax2.axhline(y=sarsa_metrics['avg_reward'], color='red', linestyle='--',
           label='Baseline (no teacher)')
ax2.set_xlabel('Teacher Accuracy')
ax2.set_ylabel('Average Reward')
ax2.set_title('SARSA: Teacher Impact Analysis')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary and Conclusions

In [ ]:
# Create summary table
summary_data = {
    'Algorithm': ['Q-learning (baseline)', 'SARSA (baseline)',
                  'Q-learning (best teacher)', 'SARSA (best teacher)'],
    'Success Rate (%)': [
        q_metrics['success_rate'],
        sarsa_metrics['success_rate'],
        q_teacher_df.loc[q_teacher_df['avg_reward'].idxmax(), 'success_rate'],
        sarsa_teacher_df.loc[sarsa_teacher_df['avg_reward'].idxmax(), 'success_rate']
    ],
    'Average Reward': [
        q_metrics['avg_reward'],
        sarsa_metrics['avg_reward'],
        q_teacher_df['avg_reward'].max(),
        sarsa_teacher_df['avg_reward'].max()
    ],
    'Learning Speed': [
        q_metrics['avg_learning_speed'],
        sarsa_metrics['avg_learning_speed'],
        q_teacher_df.loc[q_teacher_df['avg_reward'].idxmax(), 'avg_learning_speed'],
        sarsa_teacher_df.loc[sarsa_teacher_df['avg_reward'].idxmax(), 'avg_learning_speed']
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\nPerformance Summary:")
print(summary_df.to_string(index=False))

# Find best teacher configurations
best_q_config = q_teacher_df.loc[q_teacher_df['avg_reward'].idxmax()]
best_sarsa_config = sarsa_teacher_df.loc[sarsa_teacher_df['avg_reward'].idxmax()]

print("\nBest Teacher Configurations:")
print(f"Q-learning: Availability={best_q_config['availability']}, "
      f"Accuracy={best_q_config['accuracy']}")
print(f"SARSA: Availability={best_sarsa_config['availability']}, "
      f"Accuracy={best_sarsa_config['accuracy']}")

## Key Findings

### 1. **Algorithm Comparison (Baseline Performance)**

From Figures 1-3 and the summary table:

**Q-learning vs SARSA baseline performance:**
- Q-learning achieved 88.2% success rate with average reward of -11.21
- SARSA achieved 86.6% success rate with average reward of -12.80
- Q-learning slightly outperforms SARSA (approximately 1.6% difference in success rate)

**Analysis:**
Q-learning's advantage stems from its off-policy nature. It always updates based on the maximum Q-value of the next state (greedy assumption), enabling faster identification of optimal paths during exploration. SARSA, being an on-policy algorithm, considers the actual action to be taken (including exploratory actions), resulting in more conservative learning and slightly slower convergence.

### 2. **Teacher Impact Analysis**

From the heatmaps (Figures 4-5), several key observations emerge:

**Negative impact zones:**
- When accuracy ≤ 0.3, teacher guidance is generally harmful
- Worst case scenario: high availability (1.0) + low accuracy (0.1)
  - Q-learning: -152.3 (141 points worse than baseline)
  - SARSA: -198.5 (186 points worse than baseline)

**Positive impact zones:**
- When accuracy ≥ 0.7, teacher guidance becomes significantly beneficial
- Best performance occurs in high accuracy regions

**Critical point analysis:**
Figure 6 reveals that accuracy ≈ 0.5 is a critical threshold. Below this value, teacher guidance is detrimental; above it, positive effects emerge.

### 3. **Optimal Teacher Configuration**

**Q-learning optimal configuration:**
- Availability = 1.0, Accuracy = 1.0
- Average reward: 13.38 (improvement of 24.59 from baseline)
- Success rate: 98.0% (9.8% improvement)

**SARSA optimal configuration:**
- Availability = 1.0, Accuracy = 1.0
- Average reward: 14.23 (improvement of 27.03 from baseline)
- Success rate: 98.0% (11.4% improvement)

**Interesting finding:**
Counter-intuitively, for Q-learning with accuracy=1.0, availability=0.1 (13.4) performs almost identically to availability=1.0 (13.38). This suggests that for perfect teachers, occasional correct guidance is sufficient.

### 4. **Algorithm Sensitivity Analysis**

**SARSA is more sensitive to teacher guidance:**

1. **Negative sensitivity:**
   - SARSA performs worse under low-accuracy teachers (-198.5 vs -152.3)
   - Indicates that incorrect guidance is more destructive to conservative algorithms

2. **Positive sensitivity:**
   - SARSA benefits more from good teachers (improvement of 27.03 vs 24.59)
   - In high accuracy regions, SARSA's improvement curve is steeper (Figure 6, right)

**Explanation:**
SARSA's on-policy nature makes it more dependent on current policy quality. Good teacher guidance directly improves policy quality, creating a compound effect. Q-learning's off-policy nature provides some "interference resistance," making it less sensitive to variations in teacher quality.

## Conclusions

### Main Conclusions:

1. **Baseline performance:** Q-learning slightly outperforms conservative SARSA without teacher guidance due to its aggressive learning strategy.

2. **Teacher quality threshold:** A clear quality threshold exists (accuracy ≈ 0.5). Teacher guidance below this threshold is harmful, emphasizing the importance of validating expert knowledge quality in practical applications.

3. **Optimal guidance strategy:** High-quality continuous guidance (high accuracy + high availability) is most effective for both algorithms. Interestingly, occasional guidance from a perfect teacher can achieve similar results.

4. **Algorithm selection recommendations:**
   - No teacher/uncertain teacher quality: Choose Q-learning (more robust)
   - High-quality teacher available: Choose SARSA (greater benefits)

### Practical Implications:

This research provides important insights for real-world reinforcement learning applications:
- Expert accuracy must be evaluated before incorporating human guidance
- Low-quality "expert" guidance is worse than allowing agents to learn autonomously
- Different RL algorithms respond differently to external guidance, requiring appropriate algorithm selection based on available resources